In [14]:
import os
import json
import subprocess
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from groq import Groq

# Set up the Drive v3 API
SCOPES = ['https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = "C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-3a0e2f411c8d.json"

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=credentials)

In [15]:
# Set up the Groq client
from groq import Groq

client = Groq(
    api_key='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'
)

In [16]:
# Function to list files in Google Drive
def list_files(service):
    try:
        results = service.files().list(pageSize=10, fields="files(id, name)").execute()
        items = results.get('files', [])
        if not items:
            print('No files found.')
            return "No files found."
        else:
            files_list = [{'name': item['name'], 'id': item['id']} for item in items]
            print('Files:')
            for item in items:
                print(f"{item['name']} ({item['id']})")
            return files_list
    except HttpError as error:
        print(f'An error occurred: {error}')
        return f'An error occurred: {error}'

# Define the functions as strings for dynamic code generation
functions_code = """
def move_file_to_folder(service, file_id, folder_id):
    try:
        # Retrieve the existing parents to remove
        file = service.files().get(fileId=file_id, fields='parents').execute()
        previous_parents = ",".join(file.get('parents'))
        
        # Move the file to the new folder
        file = service.files().update(
            fileId=file_id,
            addParents=folder_id,
            removeParents=previous_parents,
            fields='id, parents'
        ).execute()
        print(f'Moved file {file_id} to folder {folder_id}')
    except HttpError as error:
        print(f'An error occurred: {error}')

def create_folder(service, folder_name):
    file_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    try:
        file = service.files().create(body=file_metadata, fields='id').execute()
        print(f'Folder "{folder_name}" created with ID: {file.get("id")}')
        return file.get('id')
    except HttpError as error:
        print(f'An error occurred: {error}')
"""

In [17]:
# List files in Google Drive
list_of_files = list_files(service)

Files:
rolf (1bvvzgPtAwo8hFCU1o3TqzdWY9qdxAVC5)
Artificial Intelligence - Revolutionizing the Future (1c_y7ClGRabjqg5-dzbUsve9DP6Ox_5qxv7-UfaPEDHQ)
Artificial-Intelligence-Revolutionizing-the-Future (1_V6bAUWkvkWNBAhJt6_oc7LaBsv9r5HMO361WJksqpU)
Artificial-Intelligence-Revolutionizing-the-Future (13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U)
Artificial-Intelligence-Revolutionizing-the-Future (115KLh0EgU3WOOEvW0eV4CpExNRFtflY9MnI1SrGZUFY)
Transforming-the-Future-with-Artificial-Intelligence (1oWVdSQ6D0HOXoHfQNtOrLSrci6IMdPu8rMKC5ptP_fg)
AI-Revolutionizing-the-Future (1W3SDE3a-EkyoHiMt3Bt4WIJFcYhR092-_DfX6R45MBk)
Transforming the Future Through Artificial Intelligence (1m4IbNK5Cy49ZQYgvbZQ7ftaJThxXVo11swDGZT4xVWQ)
Transforming-the-Future-of-Intelligence (1BJZSGw7FvihmLl14bFtvP0VxRU1YkbDQEGi9ymR0EZc)
Real-Time Google Docs Example (15vLJ3KUP0pVinTW1udzVolz8xGM8rAGZqPEgR_DFTys)


In [18]:
# Prepare the system message with the available functions and file list
system_msg = f"Here is a list of the files and folders in the GDrive currently: {list_of_files}\n\n"
system_msg += functions_code
system_msg += "this is the path to the service account key: "+ SERVICE_ACCOUNT_FILE + "\n \n"
system_msg += """
You are an expert at Google Cloud and a GDrive manager. Your job is to use the functions above to execute the user's wishes.
You will generate a Python script using the functions above to execute the user's wishes. WRITE THE FUNCTIONS IN THE FILE
IMPORTANT: ALSO IN THE FILE INCLUDE THESE IMPORTS : import os
import json
import subprocess
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from groq import Groq
IMPORTANT: Use the list of files for context so that you know what the file IDs corresponding to the names are.
IMPORTANT: DO NOT WRITE ANYTHING IN THE FILE UNLESS IT IS CODE OR A COMMENT AND IF IT IS COMMENT THEN IT MUST HAVE THE COMMENT HASHTAG DO NOT WRITE FILES WITHTEXT THAT WOULD CAUSE AN ERROR.
IMPORTANT: WHEN MAKING FILE PATH, ALWAYS DO DOUBLE SLASHES \\, NOT SINGLE SLASH
IMPORTANT: PLEASE WRITE THE FUNCTION DEFINITIONS FOR--> MOVE FOLDER AND CREATE FOLDER, DO NOT JUST CALL THEM OKAYYYYY!!!!
IMPORTANT: PLEASE WRITE CODE TO SHARE THE CREATED ITEMS TO THE USER via permissions
"""

In [19]:
print(system_msg)

Here is a list of the files and folders in the GDrive currently: [{'name': 'rolf', 'id': '1bvvzgPtAwo8hFCU1o3TqzdWY9qdxAVC5'}, {'name': 'Artificial Intelligence - Revolutionizing the Future', 'id': '1c_y7ClGRabjqg5-dzbUsve9DP6Ox_5qxv7-UfaPEDHQ'}, {'name': 'Artificial-Intelligence-Revolutionizing-the-Future', 'id': '1_V6bAUWkvkWNBAhJt6_oc7LaBsv9r5HMO361WJksqpU'}, {'name': 'Artificial-Intelligence-Revolutionizing-the-Future', 'id': '13xWMvDMbjaaw7VRxCyEs_7V_7o2ejgiwZE_syOr6J8U'}, {'name': 'Artificial-Intelligence-Revolutionizing-the-Future', 'id': '115KLh0EgU3WOOEvW0eV4CpExNRFtflY9MnI1SrGZUFY'}, {'name': 'Transforming-the-Future-with-Artificial-Intelligence', 'id': '1oWVdSQ6D0HOXoHfQNtOrLSrci6IMdPu8rMKC5ptP_fg'}, {'name': 'AI-Revolutionizing-the-Future', 'id': '1W3SDE3a-EkyoHiMt3Bt4WIJFcYhR092-_DfX6R45MBk'}, {'name': 'Transforming the Future Through Artificial Intelligence', 'id': '1m4IbNK5Cy49ZQYgvbZQ7ftaJThxXVo11swDGZT4xVWQ'}, {'name': 'Transforming-the-Future-of-Intelligence', 'id': '

In [20]:
# Main loop to handle user requests
while True:
    client_req = str(input("Enter your request: "))
    response = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": client_req}
        ],
        model="llama3-70b-8192",
    )
    
    generated_code = response.choices[0].message.content.strip()
    
    
    fidx = generated_code.find("```") + 3
    ridx = generated_code.rfind("```")
    generated_code = generated_code[fidx:ridx]

    print("Generated Code:")
    print(generated_code)
    
    # Save the generated code to a temporary file
    with open('temp_script.py', 'w') as f:
        f.write(generated_code)
    
    venv_path = os.path.join(os.getcwd(), '.venv')

    # Ensure the virtual environment's Python interpreter is used
    python_executable = os.path.join(venv_path, 'Scripts', 'python.exe')
    script_path = os.path.join(os.getcwd(), 'temp_script.py')
    print(script_path)
    # Execute the generated code
    result = subprocess.run([python_executable, script_path], capture_output=True, text=True)

    # Print the output of the executed code
    print("Execution Output:")
    print(result.stdout)
    if result.stderr:
        print("Execution Error:")
        print(result.stderr)
    
    # Clean up the temporary file

Generated Code:

import os
import json
import subprocess
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload

# Path to service account key file
SERVICE_ACCOUNT_FILE = 'C:\\DEV\\WebdevFolder\\realestateai-doc-mgr-3a0e2f411c8d.json'

# Create credentials using service account file
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE)

# Create the Drive API client
drive_service = build('drive', 'v3', credentials=creds)

def create_folder(service, folder_name):
    file_metadata = {
        'name': folder_name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    try:
        file = service.files().create(body=file_metadata, fields='id').execute()
        print(f'Folder "{folder_name}" created with ID: {file.get("id")}')
        return file.get('id')
    except HttpError as error:
        print(f'An error occurred: {er

KeyboardInterrupt: Interrupted by user